ERROR: tensorflow 2.2.0rc1 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0rc1 has requirement tensorflow-estimator<2.3.0,>=2.2.0rc0, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
2.1.0


Success


     |████████████████████████████████| 25.2MB 130kB/s 
     |████████████████████████████████| 6.7MB 56.4MB/s 
     |████████████████████████████████| 10.0MB 49.1MB/s 
     |████████████████████████████████| 983kB 54.7MB/s 
     |████████████████████████████████| 245kB 55.5MB/s 
     |████████████████████████████████| 1.8MB 57.1MB/s 
     |████████████████████████████████| 542kB 55.3MB/s 
     |████████████████████████████████| 1.0MB 47.5MB/s 
     |████████████████████████████████| 3.7MB 62.3MB/s 
     |████████████████████████████████| 870kB 65.4MB/s 
ERROR: google-colab 1.0.0 has requirement pandas~=0.25.0; python_version >= "3.0", but you'll have pandas 1.0.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


size of training set: 2257
size of validation set: 1502
classes: ['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']


/usr/local/lib/python3.6/dist-packages/ktrain/text/preprocessor.py:366: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn('The class_names argument is replacing the classes argument. Please update your code.')


ERROR! Session/line number was not unique in database. History logging moved to new session 59
0.11.3
2.2.5
preprocessing train...
language: en
train sequence lengths:
	mean : 308
	95percentile : 837
	99percentile : 1938


KeyError: ignored

ERROR! Session/line number was not unique in database. History logging moved to new session 60
84131840/84125825 [==============================] - 9s 0us/step


/root/.keras/datasets/aclImdb


detected encoding: utf-8
preprocessing train...
language: en
train sequence lengths:
	mean : 234
	95percentile : 598
	99percentile : 913


KeyError: ignored

ValueError: ignored